# 接口设计

`BaseAction(description=None, parser=JsonParser, enable=True)` 是所有动作应该继承的基类，它接收三个初始化参数：

- description：一个工具描述的字典，用于设置实例属性 `description`。通常不需要显式地传递这个参数，因为 `BaseAction` 的元类将查找被 `tool_api` 装饰的方法，并组装它们的 `api_description` 构造一个类属性 `__tool_description__`，如果实例化时 `description` 为空，那么该实例属性将置为 `__tool_description__。`。
- parser：BaseParser 类，用于实例化一个动作解析器校验 `description` 所描述的工具的参数。例如，`JsonParser` 会要求模型在调用工具时传入一个 JSON 格式字符串或者 Python 字典，为了让 LLM 感知到该指令，它会在 `description` 中插入一个 `parameter_description` 字段。

In [12]:
from lagent.actions import BaseAction, tool_api

In [13]:
action = BaseAction(
    {
        'name': 'bold',
        'description': 'a function used to make text bold',
        'parameters': [
            {
                'name': 'text', 'type': 'STRING', 'description': 'input content'
            }
        ],
        'required': ['text']
    }
)
action.description
# enable: 指明该动作是否生效。

{'name': 'bold',
 'description': 'a function used to make text bold',
 'parameters': [{'name': 'text',
   'type': 'STRING',
   'description': 'input content'}],
 'required': ['text'],
 'parameter_description': 'If you call this tool, you must pass arguments in the JSON format {key: value}, where the key is the parameter name.'}

# 自定义动作

一个简单工具必须实现 run 方法，而工具包则应当避免将各子API名称定义为该保留字段。

小技巧
> 对于非工具包的 Action，run 允许不被 tool_api 装饰，除非你想提示返回信息。


In [14]:
class Bold(BaseAction):

    def run(self, text: str):
        """make text bold

        Args:
            text (str): input text

        Returns:
            str: bold text
        """
        return '**' + text + '**'


# 查看默认工具描述
Bold.__tool_description__

{'name': 'Bold',
 'description': 'make text bold',
 'parameters': [{'name': 'text',
   'type': 'STRING',
   'description': 'input text'}],
 'required': ['text']}

In [15]:
class PhraseEmphasis(BaseAction):
    """a toolkit which provides different styles of text emphasis"""

    @tool_api
    def bold(self, text):
        """make text bold

        Args:
            text (str): input text

        Returns:
            str: bold text
        """
        return '**' + text + '**'

    @tool_api
    def italic(self, text):
        """make text italic

        Args:
            text (str): input text

        Returns:
            str: italic text
        """
        return '*' + text + '*'


# 查看默认工具描述
PhraseEmphasis.__tool_description__

{'name': 'PhraseEmphasis',
 'description': 'a toolkit which provides different styles of text emphasis',
 'api_list': [{'name': 'bold',
   'description': 'make text bold',
   'parameters': [{'name': 'text',
     'type': 'STRING',
     'description': 'input text'}],
   'required': ['text']},
  {'name': 'italic',
   'description': 'make text italic',
   'parameters': [{'name': 'text',
     'type': 'STRING',
     'description': 'input text'}],
   'required': ['text']}]}